In [108]:
# -*- coding: utf-8 -*-
import xml.etree.ElementTree as ET
import pandas as pd
import numpy  as np

tree = ET.parse('task.xml')
root = tree.getroot()

In [149]:
keys = []
trash = []
rows = []

for payers in root[1].findall('Плательщик'):
    try:
        id = payers.find('ЛицСч').text
    except TypeError:
        id = str(None)  ### log строка 19 + k*7 не имеет одного из ключевых реквизитов 
        continue

    try:
        name = payers.find('ФИО').text
    except TypeError:
        name = str(None)
    
    try:
        adres = payers.find('Адрес').text
    except  TypeError:
        adres = str(None)

    try:    
        sum = payers.find('Сумма').text
        try:
            sum = float(sum)
            if (sum  <= 0)  or (len(str(sum).split('.')[1]) > 2):            ### log неверный формат суммы
                sum = str(None)
            else:
                sum = str(sum)
        except ValueError:
            sum = str(None)                            ### log  неверный формат суммы
    except TypeError:
        sum = str(None)

    try:
        period = payers.find('Период').text
    except TypeError:
        period = str(None)            
        continue                                ### log строка 19 + k*7 не имеет одного из ключевых реквизитов 

    if len(period) != 6:
        period = str(None)               ###  log  неверный формат периода
    else:
        for i in (period):
            try:
                i = str(i)              #### log  неверный формат периода
            except ValueError:
                period = str(None)
                break

    if (id + period) in keys:
        trash.append(id)             ##### log  невозможно обработать запись, присутствуют дубликаты
        keys.append(id + period)
        continue
    
    keys.append(id + period)

    rows.append({"ЛицСч": id,
             "ФИО": name,
             "Адрес": adres,
             "Период": period,
             "Сумма": sum})

doc_date = root[0][0][0][1].text

df = pd.DataFrame(rows)
df.insert(0, 'Дата акт. данных', doc_date)
df.insert(0, 'Имя файла реестра', 'task')

df = df[df.ЛицСч.apply(lambda s:  bool(s not in trash))]


df.to_csv('output.csv', sep = ';', header=False)


In [150]:
df

,Имя файла реестра,Дата акт. данных,ЛицСч,ФИО,Адрес,Период,Сумма
0,task,05.10.2020,12345,Иванов Иван Иванович,"г. Киров, ул. Ленина, 1,1",092020,100.0
1,task,05.10.2020,12346,Петров Иван Иванович,"г. Киров, ул. Ленина, 1,2",092020,101.0
3,task,05.10.2020,12348,Сидоров Петр Иванович,"г. Киров, ул. Ленина, 1,4",092020,99.32
4,task,05.10.2020,12349,Сидоров Петр Некрасович,"г. Киров, ул. Ленина, 1,5",092020,None
5,task,05.10.2020,12399,Гектор Иван Некрасович,"г. Киров, ул. Ленина, 1,6",None,None


In [151]:
df['Сумма'].where(~(df.Сумма == 'None'), other='0', inplace=True)

In [152]:
df.Сумма = df.Сумма.apply(lambda s:  float(s))
df['Сумма'].where(~(df.Сумма == 0), other=None, inplace=True)

In [153]:
df

,Имя файла реестра,Дата акт. данных,ЛицСч,ФИО,Адрес,Период,Сумма
0,task,05.10.2020,12345,Иванов Иван Иванович,"г. Киров, ул. Ленина, 1,1",092020,100.00
1,task,05.10.2020,12346,Петров Иван Иванович,"г. Киров, ул. Ленина, 1,2",092020,101.00
3,task,05.10.2020,12348,Сидоров Петр Иванович,"г. Киров, ул. Ленина, 1,4",092020,99.32
4,task,05.10.2020,12349,Сидоров Петр Некрасович,"г. Киров, ул. Ленина, 1,5",092020,NaN
5,task,05.10.2020,12399,Гектор Иван Некрасович,"г. Киров, ул. Ленина, 1,6",None,NaN


In [154]:
df.to_csv('output.csv', sep = ';', header=False, encoding='windows-1251')